In [1]:
import numpy as np
import torch
import torch.nn as nn

In [63]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size
        
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

rnn = RNN(5, 32, 15)

In [64]:
learning_rate = 0.005
criterion = nn.NLLLoss()

def train(y_train, x_train):
    hidden = rnn.initHidden()

    rnn.zero_grad()

    for i in range(x_train.size()[0]):
        output, hidden = rnn(x_train[i], hidden)

    loss = criterion(output, y_train)
    loss.backward()

    # Add parameters' gradients to their values, multiplied by learning rate
    for p in rnn.parameters():
        gd = p.grad.data
        gd[gd != gd] = 0
        p.data.add_(-learning_rate, gd)

    return output, loss.item()

In [67]:
test_x = torch.rand(1000,1,5)
test_y = torch.tensor([14] ,dtype=torch.long)

train(test_y, test_x)


(tensor([[-2.8164, -2.7524, -3.0757, -2.3285, -2.9229, -3.1308, -2.6659, -2.9547,
          -2.7907, -2.8128, -2.3699, -2.4412, -2.8177, -2.7756, -2.4112]],
        grad_fn=<LogSoftmaxBackward>), 2.4112470149993896)